# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-04-24, 09:55:00

   -> Summany DataFrame : 128 rows, 20 columns

TDMDBPR : Disconnected


In [3]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250418,DB1R000100,Prepay Revenue,2025-04-18,20250409,20250418,"542,486,254","1,084,972,509","1,084,083,326","1,084,083,326",0,"11,487,390",4,170,1850
1,20250418,DB2R000100,Postpaid Revenue,2025-04-18,20250409,20250418,"840,620,664","1,681,241,328","1,679,733,746","1,679,733,746",0,"30,001,468",4,170,1830
2,20250418,DB2R010100,Postpaid Revenue B2C,2025-04-18,20250409,20250418,"815,134,153","1,630,268,305","1,628,770,254","1,628,770,254",0,"29,875,970",4,170,1830
3,20250418,DB2R020100,Postpaid Revenue B2B,2025-04-18,20250409,20250418,"25,486,511","50,973,023","50,963,492","50,963,492",0,"125,498",4,170,1830


In [4]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250418,DB1S000500,Prepay Usage Subs,2025-04-18,20250409,20250418,"113,593,067","227,186,134","227,006,751","227,006,751",0,"2,552,079",4,170,1850
1,20250418,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-04-18,20250409,20250418,"100,488,337","106,101,846","110,098,027","110,374,907",0,"2,468,195",4,170,1740
2,20250418,DB2S000500,Postpaid Active Subs,2025-04-18,20250409,20250418,"58,041,822","116,083,644","115,974,085","115,974,085",0,"1,948,294",4,170,1850
3,20250418,DB2S010500,Postpaid Active Subs B2C,2025-04-18,20250409,20250418,"51,131,502","102,263,004","102,175,893","102,175,893",0,"1,754,227",4,170,1850
4,20250418,DB2S020400,Postpaid Active Sub B2B,2025-04-18,20250409,20250418,"6,910,320","13,820,640","13,798,192","13,798,192",0,"194,067",4,170,1830


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [5]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-04-24, 09:55:03

   -> Daily DataFrame : 90 rows, 16 columns

TDMDBPR : Disconnected


In [6]:
''' DB1R000100 : Prepay Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250418,DB1R000100,Prepay Revenue,20250409,2025-04-18,"54,444,390","108,888,779","108,797,626","108,797,626",0,"1,138,984",0,0,4,170,185
1,20250418,DB1R000100,Prepay Revenue,20250410,2025-04-18,"57,122,748","114,245,496","114,154,594","114,154,594",0,"1,157,204",0,0,4,170,185
2,20250418,DB1R000100,Prepay Revenue,20250411,2025-04-18,"59,452,866","118,905,733","118,810,792","118,810,792",0,"1,205,197",0,0,4,170,185
3,20250418,DB1R000100,Prepay Revenue,20250412,2025-04-18,"59,238,338","118,476,676","118,376,257","118,376,257",0,"1,256,805",0,0,4,170,185
4,20250418,DB1R000100,Prepay Revenue,20250413,2025-04-18,"52,617,783","105,235,566","105,148,619","105,148,619",0,"1,130,314",0,0,4,170,185
5,20250418,DB1R000100,Prepay Revenue,20250414,2025-04-18,"49,106,476","98,212,951","98,138,254","98,138,254",0,"1,106,194",0,0,4,170,185
6,20250418,DB1R000100,Prepay Revenue,20250415,2025-04-18,"49,605,471","99,210,942","99,121,423","99,121,423",0,"1,091,940",0,0,4,170,185
7,20250418,DB1R000100,Prepay Revenue,20250416,2025-04-18,"53,196,521","106,393,042","106,304,710","106,304,710",0,"1,130,089",0,0,4,170,185
8,20250418,DB1R000100,Prepay Revenue,20250417,2025-04-18,"54,043,511","108,087,022","107,999,932","107,999,932",0,"1,151,668",0,0,4,170,185
9,20250418,DB1R000100,Prepay Revenue,20250418,2025-04-18,"53,658,151","107,316,302","107,231,118","107,231,118",0,"1,118,994",0,0,4,170,185


In [7]:
''' DB2R000100 : Postpaid Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250418,DB2R000100,Postpaid Revenue,20250409,2025-04-18,"87,456,680","174,913,360","174,756,893","174,756,893",0,"3,054,996",0,0,4,170,183
1,20250418,DB2R000100,Postpaid Revenue,20250410,2025-04-18,"92,927,038","185,854,077","185,690,309","185,690,309",0,"3,315,314",0,0,4,170,183
2,20250418,DB2R000100,Postpaid Revenue,20250411,2025-04-18,"86,697,448","173,394,897","173,228,032","173,228,032",0,"3,123,773",0,0,4,170,183
3,20250418,DB2R000100,Postpaid Revenue,20250412,2025-04-18,"85,714,258","171,428,516","171,266,317","171,266,317",0,"3,120,846",0,0,4,170,183
4,20250418,DB2R000100,Postpaid Revenue,20250413,2025-04-18,"84,672,588","169,345,176","169,192,961","169,192,961",0,"3,174,570",0,0,4,170,183
5,20250418,DB2R000100,Postpaid Revenue,20250414,2025-04-18,"78,494,695","156,989,390","156,835,022","156,835,022",0,"2,844,605",0,0,4,170,183
6,20250418,DB2R000100,Postpaid Revenue,20250415,2025-04-18,"80,393,629","160,787,258","160,639,597","160,639,597",0,"2,870,098",0,0,4,170,183
7,20250418,DB2R000100,Postpaid Revenue,20250416,2025-04-18,"80,395,169","160,790,337","160,653,275","160,653,275",0,"2,945,108",0,0,4,170,183
8,20250418,DB2R000100,Postpaid Revenue,20250417,2025-04-18,"81,062,686","162,125,371","161,987,209","161,987,209",0,"2,763,153",0,0,4,170,183
9,20250418,DB2R000100,Postpaid Revenue,20250418,2025-04-18,"82,806,473","165,612,946","165,484,133","165,484,133",0,"2,789,006",0,0,4,170,183


In [8]:
''' DB2R010100 : Postpaid Revenue B2C '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250418,DB2R010100,Postpaid Revenue B2C,20250409,2025-04-18,"85,153,510","170,307,021","170,151,203","170,151,203",0,"3,044,648",0,0,4,170,183
1,20250418,DB2R010100,Postpaid Revenue B2C,20250410,2025-04-18,"88,491,912","176,983,824","176,820,381","176,820,381",0,"3,303,899",0,0,4,170,183
2,20250418,DB2R010100,Postpaid Revenue B2C,20250411,2025-04-18,"84,986,570","169,973,139","169,809,187","169,809,187",0,"3,100,464",0,0,4,170,183
3,20250418,DB2R010100,Postpaid Revenue B2C,20250412,2025-04-18,"83,901,943","167,803,885","167,643,915","167,643,915",0,"3,101,922",0,0,4,170,183
4,20250418,DB2R010100,Postpaid Revenue B2C,20250413,2025-04-18,"82,958,015","165,916,030","165,763,918","165,763,918",0,"3,159,312",0,0,4,170,183
5,20250418,DB2R010100,Postpaid Revenue B2C,20250414,2025-04-18,"77,170,485","154,340,971","154,188,655","154,188,655",0,"2,832,045",0,0,4,170,183
6,20250418,DB2R010100,Postpaid Revenue B2C,20250415,2025-04-18,"77,192,352","154,384,705","154,237,155","154,237,155",0,"2,859,774",0,0,4,170,183
7,20250418,DB2R010100,Postpaid Revenue B2C,20250416,2025-04-18,"78,332,139","156,664,278","156,527,703","156,527,703",0,"2,935,394",0,0,4,170,183
8,20250418,DB2R010100,Postpaid Revenue B2C,20250417,2025-04-18,"77,712,247","155,424,494","155,286,689","155,286,689",0,"2,755,886",0,0,4,170,183
9,20250418,DB2R010100,Postpaid Revenue B2C,20250418,2025-04-18,"79,234,979","158,469,958","158,341,446","158,341,446",0,"2,782,626",0,0,4,170,183


In [9]:
''' DB2R020100 : Postpaid Revenue B2B '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250418,DB2R020100,Postpaid Revenue B2B,20250409,2025-04-18,"2,303,170","4,606,339","4,605,690","4,605,690",0,"10,348",0,0,4,170,183
1,20250418,DB2R020100,Postpaid Revenue B2B,20250410,2025-04-18,"4,435,126","8,870,253","8,869,927","8,869,927",0,"11,415",0,0,4,170,183
2,20250418,DB2R020100,Postpaid Revenue B2B,20250411,2025-04-18,"1,710,879","3,421,757","3,418,844","3,418,844",0,"23,309",0,0,4,170,183
3,20250418,DB2R020100,Postpaid Revenue B2B,20250412,2025-04-18,"1,812,316","3,624,631","3,622,402","3,622,402",0,"18,924",0,0,4,170,183
4,20250418,DB2R020100,Postpaid Revenue B2B,20250413,2025-04-18,"1,714,573","3,429,146","3,429,043","3,429,043",0,"15,258",0,0,4,170,183
5,20250418,DB2R020100,Postpaid Revenue B2B,20250414,2025-04-18,"1,324,210","2,648,419","2,646,367","2,646,367",0,"12,559",0,0,4,170,183
6,20250418,DB2R020100,Postpaid Revenue B2B,20250415,2025-04-18,"3,201,277","6,402,553","6,402,442","6,402,442",0,"10,324",0,0,4,170,183
7,20250418,DB2R020100,Postpaid Revenue B2B,20250416,2025-04-18,"2,063,029","4,126,059","4,125,571","4,125,571",0,"9,714",0,0,4,170,183
8,20250418,DB2R020100,Postpaid Revenue B2B,20250417,2025-04-18,"3,350,439","6,700,877","6,700,520","6,700,520",0,"7,267",0,0,4,170,183
9,20250418,DB2R020100,Postpaid Revenue B2B,20250418,2025-04-18,"3,571,494","7,142,988","7,142,686","7,142,686",0,"6,380",0,0,4,170,183


In [10]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]